# Fermi-Hubbard Model

In [ ]:
using Pkg; Pkg.activate()
using KadanoffBaym
using LinearAlgebra
using UnPack

In [ ]:
using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")

## Model

### Hamiltonian:

$$
\begin{align}\begin{split}
    \hat{H} &= - J \left(f^{\dagger}_1 f^\phantom{\dagger}_2 + f^{\dagger}_2 f^\phantom{\dagger}_1\right) - J \left(c^{\dagger}_1 c^\phantom{\dagger}_2 + c^{\dagger}_2 c^\phantom{\dagger}_1\right) + U \left( f^{\dagger}_1 f^\phantom{\dagger}_1 c^{\dagger}_1 c^\phantom{\dagger}_1  +  f^{\dagger}_2 f^\phantom{\dagger}_2 c^{\dagger}_2 c^\phantom{\dagger}_2 \right)
\end{split}\end{align}
$$

$$
    G^>_{ij}(t, t') = -i \langle c^\phantom{\dagger}_i(t) c^{\dagger}_j(t') \rangle\\
    F^>_{ij}(t, t') = -i \langle f^\phantom{\dagger}_i(t) f^{\dagger}_j(t') \rangle
$$

$$
    \Sigma^{\mathrm{HF}}_{c,\,ij}(t, t') = {\mathrm{i}}\delta_{ij}\delta(t - t') F^<_{ii}(t, t)\\
    \Sigma^{\mathrm{HF}}_{f,\,ij}(t, t') = {\mathrm{i}}\delta_{ij}\delta(t - t') G^<_{ii}(t, t)
$$

$$
    \Sigma^{\mathrm{NCA}}_{c,\,ij}(t, t') = {\mathrm{i}\cdot (-\mathrm{i}) \cdot  U^2} G_{ij}(t, t') F_{ij}(t, t') F_{ji}(t', t)\\
    \Sigma^{\mathrm{NCA}}_{f,\,ij}(t, t') = {\mathrm{i}\cdot (-\mathrm{i}) \cdot  U^2} F_{ij}(t, t') G_{ij}(t, t') G_{ji}(t', t)
$$

## Solving

In [ ]:
Base.@kwdef struct FermiHubbardModel
    U::Float64
    H = ComplexF64[0 -1 0 0; -1 0 0 0; 0 0 0 -1; 0 0 -1 0]
end

In [ ]:
struct FermiHubbardData{T}
    GL::T
    GG::T
    FL::T
    FG::T

    ΣNCA_c_L::T
    ΣNCA_c_G::T
    ΣNCA_f_L::T
    ΣNCA_f_G::T

    # Initialize problem
    function FermiHubbardData(GL::T, GG::T, FL::T, FG::T) where {T}
        new{T}(GL, GG, FL, FG, zero(GL), zero(GG), zero(FL), zero(FG))
    end
end

In [ ]:
function fv!(model, data, out, times, t, t′)
    @unpack GL, GG, FL, FG, ΣNCA_c_L, ΣNCA_c_G, ΣNCA_f_L, ΣNCA_f_G = data
    @unpack H, U = model

    # real-time collision integral
    ∫dt(i, j, A, B) =
        sign(j - i) * integrate(times[min(i, j):max(i, j)],[A[t, s] * B[s, t′] for s = min(i, j):max(i, j)])

    # normal operator ordering
    ΣHF_c(t, t′) = 1.0im * U * [FL[1, 1, t, t] 0; 0 FL[2, 2, t, t]]
    ΣHF_f(t, t′) = 1.0im * U * [GL[1, 1, t, t] 0; 0 GL[2, 2, t, t]]

    out[1] = -1.0im * ((H[1:2, 1:2] + ΣHF_c(t, t′)) * GL[t, t′] + 
            ∫dt(1, t, ΣNCA_c_G, GL) - ∫dt(1, t, ΣNCA_c_L, GL) + ∫dt(1, t′, ΣNCA_c_L, GL) -
            ∫dt(1, t′, ΣNCA_c_L, GG)
        )

    out[2] = -1.0im * ((H[1:2, 1:2] + ΣHF_c(t, t′)) * GG[t, t′] + 
            ∫dt(1, t, ΣNCA_c_G, GG) - ∫dt(1, t, ΣNCA_c_L, GG) + ∫dt(1, t′, ΣNCA_c_G, GL) -
            ∫dt(1, t′, ΣNCA_c_G, GG)
        )

    out[3] = -1.0im * ((H[3:4, 3:4] + ΣHF_f(t, t′)) * FL[t, t′] + 
            ∫dt(1, t, ΣNCA_f_G, FL) - ∫dt(1, t, ΣNCA_f_L, FL) + ∫dt(1, t′, ΣNCA_f_L, FL) -
            ∫dt(1, t′, ΣNCA_f_L, FG)
        )

    out[4] = -1.0im * ((H[3:4, 3:4] + ΣHF_f(t, t′)) * FG[t, t′] +
            ∫dt(1, t, ΣNCA_f_G, FG) - ∫dt(1, t, ΣNCA_f_L, FG) + ∫dt(1, t′, ΣNCA_f_G, FL) -
            ∫dt(1, t′, ΣNCA_f_G, FG)
        )

    return out
end

function fd!(model, data, out, times, t, t′)
    fv!(model, data, out, times, t, t)
    out .-= adjoint.(out)
end

function self_energies!(model, data, times, t, t′)
    @unpack GL, GG, FL, FG, ΣNCA_c_L, ΣNCA_c_G, ΣNCA_f_L, ΣNCA_f_G = data
    @unpack U = model

    if (n = size(GL, 3)) > size(ΣNCA_c_L, 3)
        resize!(ΣNCA_c_L, n)
        resize!(ΣNCA_c_G, n)
        resize!(ΣNCA_f_L, n)
        resize!(ΣNCA_f_G, n)
    end

    ΣNCA_c_L[t, t′] = 1.0im .* (-1.0im .* U^2) .* GL[t, t′] .* FL[t, t′] .* transpose(FG[t′, t])
    ΣNCA_c_G[t, t′] = 1.0im .* (-1.0im .* U^2) .* GG[t, t′] .* FG[t, t′] .* transpose(FL[t′, t])

    ΣNCA_f_L[t, t′] = 1.0im .* (-1.0im .* U^2) .* FL[t, t′] .* GL[t, t′] .* transpose(GG[t′, t])
    ΣNCA_f_G[t, t′] = 1.0im .* (-1.0im .* U^2) .* FG[t, t′] .* GG[t, t′] .* transpose(GL[t′, t])
end

In [ ]:
# quantum numbers
dim = 2

# Define your Green functions at (t0, t0) – time arguments at the end
GL = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), SkewHermitian)
GG = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), SkewHermitian)
FL = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), SkewHermitian)
FG = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), SkewHermitian)

# Initial condition
N_c = [0.5, 0.1]
N_f = [0.3, 0.7]

N_c = [0., 1]
N_f = [1., 0]


GL[1, 1] = 1.0im * diagm(N_c)
GG[1, 1] = -1.0im * (I - diagm(N_c))
FL[1, 1] = 1.0im * diagm(N_f)
FG[1, 1] = -1.0im * (I - diagm(N_f))

data = FermiHubbardData(GL, GG, FL, FG)
model = FermiHubbardModel(U = 0.5)

tmax = 100;

In [ ]:
function integrate(x::AbstractVector, y::AbstractVector)
    if isone(length(x))
        return zero(first(y))
    end

    @inbounds retval = (x[2] - x[1]) * (y[1] + y[2])
    @inbounds @fastmath @simd for i = 2:(length(y)-1)
        retval += (x[i+1] - x[i]) * (y[i] + y[i+1])
    end
    return 1 // 2 * retval
end

In [ ]:
sol = kbsolve!(
    (x...) -> fv!(model, data, x...),
    (x...) -> begin
        println(" t: $(x[2][x[3]])")
        fd!(model, data, x...)
    end,
    [data.GL, data.GG, data.FL, data.FG],
    (0.0, tmax);
    callback = (x...) -> self_energies!(model, data, x...),
    atol = 1e-6,
    rtol = 1e-4,
);

## Example plots

In [ ]:
sol.t

In [ ]:
xpad = 8
ypad = 5

figure(figsize = (8, 3))

ax = subplot(121)
plot(sol.t, [imag(data.GL[1, 1, k, k]) for k = 1:length(sol.t)],  ls = "-", c = "C0", label = "\$ \\sigma=\\;\\uparrow\$")

plot(sol.t, [imag(data.FL[1, 1, k, k]) for k = 1:length(sol.t)],
    label = "\$ \\sigma=\\;\\downarrow\$", ls = "--", c = "C1")

xlim(0, 20)
# ax.set_xticks([0, 5, 10])
ylim(0, 1)
xlabel("\$J t\$")
ylabel("\$\\mathrm{Im}\\; G^<_{11, \\sigma}(t, t)\$")
# ylabel("\$\\mathrm{Im}\\; G^<_{11, \\uparrow}(t, t)\$")
ax.xaxis.set_tick_params(pad = xpad)
ax.yaxis.set_tick_params(pad = ypad)
# ax.set_axisbelow(false)
# ticklabel_format(axis = "y", style = "sci", scilimits = (-0, 0))
ax.legend(loc = "best", handlelength = 1.9, frameon = false, borderpad = 0, labelspacing = 0.25)

ax = subplot(122)

plot(sol.t, [imag(data.GL[1, 1, k, k]) for k = 1:length(sol.t)],  ls = "-", c = "C0" )#label = "\$ \\sigma=\\;\\uparrow\$")
# plot(sol.t, [imag(data.GL[2, 2, k, k]) for k = 1:length(sol.t)],  ls = "-", c = "C0" )#label = "\$ \\sigma=\\;\\uparrow\$")

# plot(sol.t, [imag(data.FL[2, 2, k, k]) for k = 1:length(sol.t)],
#     label = "\$ \\sigma=\\;\\downarrow\$", ls = "--", c = "C1")
xlim(0, tmax)
ylim(0, 1)
ax.set_xticks([0, 50, 100])
ax.set_yticklabels([])
xlabel("\$J t\$")
# ylabel("\$\\mathrm{Im}\\; G^<_{22, \\sigma}(t, t)\$", labelpad = 16)
# ylabel("\$\\mathrm{Im}\\; G^<_{22, \\uparrow}(t, t)\$", labelpad = 16)
ylabel("\$\\mathrm{Im}\\; G^<_{11, \\uparrow}(t, t)\$", labelpad = 16)
ax.xaxis.set_tick_params(pad = xpad)
ax.yaxis.set_tick_params(pad = ypad)
ax.set_axisbelow(false)
# ticklabel_format(axis = "y", style = "sci", scilimits = (-0, 0))
# ax.legend( loc = "best", handlelength = 1.9, frameon = false, borderpad = 0, labelspacing = 0.25)
ax.yaxis.set_label_position("right")

tight_layout(pad = 0.1, w_pad = 0.5, h_pad = 0)
savefig("fermi_hubbard_T.pdf")

In [ ]:
xpad = 8
ypad = 5

figure(figsize = (3, 2))

ax = subplot(111)
plot(sol.t, [(sum(data.GL[i, i, k, k] for i = 1:2) |> imag) .+ (sum(data.FL[i, i, k, k] for i = 1:2) |> imag) for k = 1:length(sol.t)] 
    .- (N_c[1] + N_c[2] + N_f[1] + N_f[2])  .|> abs, 
    label = "\$ c \$", ls = "-", c = "k")
xlim(0, tmax)
yticks([k for k in 0:3] .* 1e-15)
ylim([0.0, 3] .* 1e-15)
ylabel("\$ n_{\\mathrm{tot}}(t) - n_0\$", labelpad = 16)

tight_layout(pad = 0.0, w_pad = 0.0, h_pad = 0)
savefig("fermi_hubbard_example_T_inset.pdf")

In [ ]:
using FFTW, Interpolations

In [ ]:
idx = 1
ρτ, (τs, ts) = wigner_transform_itp(FG[idx, idx, :, :] - FL[idx, idx, :, :], sol.t, fourier=false);
ρω, (ωs, ts) = wigner_transform_itp(FG[idx, idx, :, :] - FL[idx, idx, :, :], sol.t, fourier=true);

In [ ]:
idx = 1
ρτ, (τs, ts) = wigner_transform_itp(GG[idx, idx, :, :] - GL[idx, idx, :, :], sol.t, fourier=false);
ρω, (ωs, ts) = wigner_transform_itp(GG[idx, idx, :, :] - GL[idx, idx, :, :], sol.t, fourier=true);

In [ ]:
t_scale = 1
ω_scale = 1;

In [ ]:
xpad = 8
ypad = 5

center = floor(length(ts) / 2) |> Int

figure(figsize=(8, 3))

ax = subplot(121)
plot(t_scale * τs, -ρτ[:, center] |> imag, ls="-", c="C0", lw=1.5)
ax.set_xlabel("\$J \\tau\$")
ax.set_xlim(-t_scale * tmax / 2, t_scale * tmax / 2)
ax.set_ylim(-1.0, 1.0)
ax.set_xticks(t_scale .* [-tmax/2, -tmax/4, 0, tmax/4, tmax/2])
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.set_ylabel("\$  A_{11, \\uparrow}(\\tau, T) \$")

plot_kb = ρω[:, center] |> real

ax = subplot(122)
plot(ω_scale * ωs, plot_kb, "-", c="C0", lw=1.5)

ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.set_xlabel("\$\\omega/J\$")
ax.set_ylabel("\$ A_{11,\\uparrow}(\\omega, T)\$", labelpad=16)
ax.set_xlim(5 .* (-1, 1))
ax.set_ylim(0, 10)
ax.yaxis.set_label_position("right")
# ax.legend(loc="best", handlelength=1.4, frameon=false, borderpad=0, labelspacing=0.25)

tight_layout(pad=0.1, w_pad=0.5, h_pad=0)


savefig("fermi_hubbard_example_tau_omega.pdf")

In [ ]:
function meshgrid(xin,yin)
  nx=length(xin)
  ny=length(yin)
  xout=zeros(ny,nx)
  yout=zeros(ny,nx)
  for jx=1:nx
      for ix=1:ny
          xout[ix,jx]=xin[jx]
          yout[ix,jx]=yin[ix]
      end
  end
  return (x=xout, y=yout)
end

Y, X = meshgrid(sol.t, sol.t);

In [ ]:
X_gauss = X
Y_gauss = Y
FL_gauss = data.FL
FG_gauss = data.FG;

In [ ]:
cmap = "gist_heat";

In [ ]:
figure(figsize=(7, 3))

ax = subplot(121) # plt.gca()
heatmap = ax.pcolormesh(t_scale * X_gauss, Y_gauss, imag(FL_gauss.data[1, 1, :, :]) .- imag(FG_gauss.data[1, 1, :, :]), cmap=cmap, rasterized=true) # vmin=vmin, vmax=vmax,
heatmap.set_edgecolor("face")
ax.set_aspect("equal")
cbar = colorbar(mappable=heatmap)
cbar.formatter.set_powerlimits((0, 0))
ax.set_xlabel("\$J t\$")
ax.set_ylabel("\$J t'\$")
ax.set_xlim(0, t_scale * tmax)
ax.set_ylim(0, t_scale * tmax)
ax.set_xticks(t_scale .* [0, tmax/2, tmax])
ax.set_yticks(t_scale .* [0, tmax/2, tmax])

ax = subplot(122)
heatmap = ax.pcolormesh(t_scale * X, Y, imag(data.FL[1, 1, :, :]) .- imag(data.FG[1, 1, :, :]), cmap=cmap, rasterized=true) # vmin=vmin, vmax=vmax,
heatmap.set_edgecolor("face")
ax.set_aspect("equal")
cbar = colorbar(mappable=heatmap)
cbar.formatter.set_powerlimits((0, 0))
ax.set_xlabel("\$J t\$")
# ax.set_ylabel("\$\\lambda t'\$")
ax.set_xlim(0, t_scale * tmax)
ax.set_ylim(0, t_scale * tmax)
ax.set_xticks(t_scale .* [0, tmax/2, tmax])
ax.set_yticks(t_scale .* [0, tmax/2, tmax])
ax.set_yticklabels([])

tight_layout(pad=0.75, w_pad=0.25, h_pad=0)

# savefig("fermi_hubbard_example_two_times.pdf")

## Testing

In [ ]:
figure(figsize=(5, 3))
ax = plt.gca()
heatmap = ax.pcolormesh(t_scale * X, Y, imag(GL.data[1, 1, :, :]) .- imag(GG.data[1, 1, :, :]), cmap=cmap, rasterized=true) # vmin=vmin, vmax=vmax,
heatmap.set_edgecolor("face")
ax.set_aspect("equal")
# ax.set_xlabel("\$\\lambda T\$")
# ax.set_ylabel("\$\\omega\$")
# ax.set_xlim(0, t_scale * T)
# ax.set_xticks([0, t_scale * T/2, t_scale * T])
# ax.set_yticks([-20, -10, 0, 10, 20])
# ax.set_ylim(-20, 20)
colorbar(mappable=heatmap)
tight_layout(pad=0.0, w_pad=0, h_pad=0)